In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark import SparkContext

spark = SparkSession.builder \
    .appName("project")\
    .master("local[4]")\
    .config("spark.driver.memory","8G")\
    .config("spark.driver.maxResultSize", "2G") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.11:2.4.4")\
    .config("spark.kryoserializer.buffer.max", "500m")\
    .getOrCreate()

sc = spark.sparkContext
sc

<SparkContext master=local[4] appName=project>

In [2]:
spark

In [3]:
from sparknlp.base import Finisher, DocumentAssembler
from sparknlp.annotator import (Tokenizer, Normalizer, 
                                LemmatizerModel, StopWordsCleaner)
from pyspark.ml import Pipeline

In [4]:
import nltk
nltk.download('stopwords')
nltk.download('words')

from nltk.corpus import stopwords

eng_stopwords = stopwords.words('english')
eng_stopwords.append('xxxx')

[nltk_data] Downloading package stopwords to /home/hadoop/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package words to /home/hadoop/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


In [5]:
documentAssembler = DocumentAssembler() \
    .setInputCol('context') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

# note normalizer defaults to changing all words to lowercase.
# Use .setLowercase(False) to maintain input case.
normalizer = Normalizer() \
    .setInputCols(['token']) \
    .setOutputCol('normalized') \
    .setLowercase(True)

# note that lemmatizer needs a dictionary. So I used the pre-trained
# model (note that it defaults to english)
lemmatizer = LemmatizerModel.pretrained() \
    .setInputCols(['normalized']) \
    .setOutputCol('lemma') \

stopwords_cleaner = StopWordsCleaner() \
    .setInputCols(['lemma']) \
    .setOutputCol('clean_lemma') \
    .setCaseSensitive(False) \
    .setStopWords(eng_stopwords)

# finisher converts tokens to human-readable output
finisher = Finisher() \
    .setInputCols(['clean_lemma']) \
    .setCleanAnnotations(False)

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]


In [6]:
pipeline = Pipeline() \
    .setStages([
        documentAssembler,
        tokenizer,
        normalizer,
        lemmatizer,
        stopwords_cleaner,
        finisher
    ])

In [7]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

schema = StructType([
    StructField("article_type", StringType(), True),
    StructField("np1", StringType(), True),
    StructField("np2", StringType(), True),
    StructField("context", StringType(), True),
    StructField("source", StringType(), True),
    StructField("category", StringType(), True),
    StructField("location", StringType(), True),
    StructField("time", StringType(), True),])

In [8]:
df = spark.read.csv("s3://anly502project/data/part-r-00000",sep = "\t",header=False,schema=schema)
df_2 = spark.read.csv("s3://anly502project/data/part-r-00001",sep = "\t",header=False,schema=schema)

In [43]:
#### Data Schema
df.printSchema()
df_2.printSchema()

root
 |-- article_type: string (nullable = true)
 |-- np1: string (nullable = true)
 |-- np2: string (nullable = true)
 |-- context: string (nullable = true)
 |-- source: string (nullable = true)
 |-- category: string (nullable = true)
 |-- location: string (nullable = true)
 |-- time: string (nullable = true)

root
 |-- article_type: string (nullable = true)
 |-- np1: string (nullable = true)
 |-- np2: string (nullable = true)
 |-- context: string (nullable = true)
 |-- source: string (nullable = true)
 |-- category: string (nullable = true)
 |-- location: string (nullable = true)
 |-- time: string (nullable = true)



In [44]:
import functools 

def unionAll(dfs):
    return functools.reduce(lambda df1,df2: df1.union(df2.select(df1.columns)), dfs) 

In [45]:
unioned_df = unionAll([df, df_2])

In [ ]:
#### show combined
unioned_df.show(10)

In [13]:
from pyspark.sql.functions import split
split_col = split(unioned_df['category'], ',')
unioned_df = unioned_df.withColumn('category', split_col.getItem(0))

In [14]:
split_col_2 = split(unioned_df['category'], ' ')
unioned_df = unioned_df.withColumn('category', split_col_2.getItem(0))

In [15]:
split_col_3 = split(unioned_df['category'], '_')
unioned_df = unioned_df.withColumn('category', split_col_3.getItem(0))

In [16]:
split_col_4 = split(unioned_df['category'], '-')
unioned_df = unioned_df.withColumn('category', split_col_4.getItem(0))

In [17]:
#### Show data frame after filtering the category
unioned_df.show(10)

+------------+---------------+---+--------------------+-------------+---------+--------------------+-----+
|article_type|            np1|np2|             context|       source| category|            location| time|
+------------+---------------+---+--------------------+-------------+---------+--------------------+-----+
|     article|    Dark Knight|  E|  arg1 and Wall arg2|             | intlnews|      , kerala india|14299|
|     article|    Carotenoids|  E|arg1 and caroteno...|             |  topstor|                   ,|14660|
|     article|    Communities|  E|arg1 mobilised in...|             | politics|                   ,|14026|
|     article|    Carotenoids|  E|arg1 and caroteno...|             |  topstor|                   ,|14660|
|            |     Coast bias|  E|arg2 is for East ...|             |   sports| columbus, ohio u...|13956|
|     article|Commerce office|  E|arg1 at DDD Linco...|             |  topstor| canton, ohio uni...|14363|
|     article| 75-minute mark|  E|arg

In [18]:
df_pip = pipeline.fit(unioned_df).transform(unioned_df)

In [19]:
df_pip.show()

+------------+---------------+---+--------------------+-------------+-------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|article_type|            np1|np2|             context|       source|     category|            location| time|            document|               token|          normalized|               lemma|         clean_lemma|finished_clean_lemma|
+------------+---------------+---+--------------------+-------------+-------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|     article|    Dark Knight|  E|  arg1 and Wall arg2|             |     intlnews|      , kerala india|14299|[[document, 0, 17...|[[token, 0, 3, ar...|[[token, 0, 2, ar...|[[token, 0, 2, ar...|[[token, 0, 2, ar...|    [arg, wall, arg]|
|     article|    Carotenoids|  E|arg1 and caroteno.

In [20]:
from pyspark.sql.functions import explode, col

context_words = df_pip.withColumn("exploded_text", explode(col("finished_clean_lemma")))

In [21]:
context_words.columns

['article_type',
 'np1',
 'np2',
 'context',
 'source',
 'category',
 'location',
 'time',
 'document',
 'token',
 'normalized',
 'lemma',
 'clean_lemma',
 'finished_clean_lemma',
 'exploded_text']

In [22]:
counts = context_words.groupby('finished_clean_lemma').count()

In [23]:
#counts_pd = counts.toPandas()

Py4JJavaError: An error occurred while calling o478.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 5.0 failed 1 times, most recent failure: Lost task 1.0 in stage 5.0 (TID 17, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$dfAssembleNoExtras$1: (string) => array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:636)
	at scala.collection.Iterator$$anon$12.hasNext(Iterator.scala:440)
	at scala.collection.Iterator$JoinIterator.hasNext(Iterator.scala:212)
	at scala.collection.Iterator$$a

In [24]:
df_pip.select('finished_clean_lemma').show(20, False)

+-------------------------------------+
|finished_clean_lemma                 |
+-------------------------------------+
|[arg, wall, arg]                     |
|[arg, carotenoid, plus, vitamin, arg]|
|[arg, mobilise, action, arg]         |
|[arg, carotenoid, plus, vitamin, arg]|
|[arg, east, arg]                     |
|[arg, ddd, lincoln, way, arg]        |
|[arg, pulsate, group, arg]           |
|[arg, past, arg]                     |
|[arg, train, arg]                    |
|[arg, line, service, arg]            |
|[arg, mindstrong, rate, arg]         |
|[arg, line, stop, run, past, arg]    |
|[arg, line, stop, run, past, arg]    |
|[arg, line, beyond, arg]             |
|[arg, line, beyond, arg]             |
|[arg, cape, cold, snap, kill, arg]   |
|[arg, rate, arg]                     |
|[arg, arg]                           |
|[arg, street, band, since, arg]      |
|[arg, arg]                           |
+-------------------------------------+
only showing top 20 rows



In [25]:
df_pip.select('np1').show(20,False)

+---------------+
|np1            |
+---------------+
|Dark Knight    |
|Carotenoids    |
|Communities    |
|Carotenoids    |
|Coast bias     |
|Commerce office|
|75-minute mark |
|Brigham Circle |
|Brigham Circle |
|Brigham Circle |
|Drill Sergeant |
|Brigham Circle |
|Brigham Circle |
|Brigham Circle |
|Brigham Circle |
|Cold           |
|Cook           |
|Boob           |
|2002           |
|Boob           |
+---------------+
only showing top 20 rows



In [26]:
cols = ['np1', 'finished_clean_lemma', 'category']    

In [27]:
df_1_new = df_pip.withColumn("np1",col('np1')).select(cols)
df_2_new = df_pip.withColumn("finished_clean_lemma", col('finished_clean_lemma')).select(cols)
result = df_1_new.union(df_2_new)

In [42]:
result.show()

KeyboardInterrupt: 

In [29]:
a = result.select(split(col("np1"),",")).alias("np1_1")

In [30]:
df_new = result.withColumn("finished_clean_lemma",col('finished_clean_lemma')).select(cols)

In [31]:
df_new = result.withColumn('np1_1',split(col("np1"),","))
df_new.show()

+---------------+--------------------+-------------+-----------------+
|            np1|finished_clean_lemma|     category|            np1_1|
+---------------+--------------------+-------------+-----------------+
|    Dark Knight|    [arg, wall, arg]|     intlnews|    [Dark Knight]|
|    Carotenoids|[arg, carotenoid,...|      topstor|    [Carotenoids]|
|    Communities|[arg, mobilise, a...|     politics|    [Communities]|
|    Carotenoids|[arg, carotenoid,...|      topstor|    [Carotenoids]|
|     Coast bias|    [arg, east, arg]|       sports|     [Coast bias]|
|Commerce office|[arg, ddd, lincol...|      topstor|[Commerce office]|
| 75-minute mark|[arg, pulsate, gr...|      topstor| [75-minute mark]|
| Brigham Circle|    [arg, past, arg]|      topstor| [Brigham Circle]|
| Brigham Circle|   [arg, train, arg]|    localnews| [Brigham Circle]|
| Brigham Circle|[arg, line, servi...|      topstor| [Brigham Circle]|
| Drill Sergeant|[arg, mindstrong,...|      topstor| [Drill Sergeant]|
| Brig

In [32]:
from pyspark.sql.functions import lit, array, array_union

df2 = df_new.withColumn("finished_clean_lemma", array_union("finished_clean_lemma", col('np1_1')))
df2.show()

+---------------+--------------------+-------------+-----------------+
|            np1|finished_clean_lemma|     category|            np1_1|
+---------------+--------------------+-------------+-----------------+
|    Dark Knight|[arg, wall, Dark ...|     intlnews|    [Dark Knight]|
|    Carotenoids|[arg, carotenoid,...|      topstor|    [Carotenoids]|
|    Communities|[arg, mobilise, a...|     politics|    [Communities]|
|    Carotenoids|[arg, carotenoid,...|      topstor|    [Carotenoids]|
|     Coast bias|[arg, east, Coast...|       sports|     [Coast bias]|
|Commerce office|[arg, ddd, lincol...|      topstor|[Commerce office]|
| 75-minute mark|[arg, pulsate, gr...|      topstor| [75-minute mark]|
| Brigham Circle|[arg, past, Brigh...|      topstor| [Brigham Circle]|
| Brigham Circle|[arg, train, Brig...|    localnews| [Brigham Circle]|
| Brigham Circle|[arg, line, servi...|      topstor| [Brigham Circle]|
| Drill Sergeant|[arg, mindstrong,...|      topstor| [Drill Sergeant]|
| Brig

In [33]:
df2.select('finished_clean_lemma').show()

+--------------------+
|finished_clean_lemma|
+--------------------+
|[arg, wall, Dark ...|
|[arg, carotenoid,...|
|[arg, mobilise, a...|
|[arg, carotenoid,...|
|[arg, east, Coast...|
|[arg, ddd, lincol...|
|[arg, pulsate, gr...|
|[arg, past, Brigh...|
|[arg, train, Brig...|
|[arg, line, servi...|
|[arg, mindstrong,...|
|[arg, line, stop,...|
|[arg, line, stop,...|
|[arg, line, beyon...|
|[arg, line, beyon...|
|[arg, cape, cold,...|
|   [arg, rate, Cook]|
|         [arg, Boob]|
|[arg, street, ban...|
|         [arg, Boob]|
+--------------------+
only showing top 20 rows



In [34]:
cate = df2.select('category')

In [35]:
category = cate.distinct()

In [36]:
category.show()

KeyboardInterrupt: 

In [38]:
def term_frequency(BoW_dict):
    tot_words = sum(BoW_dict.values())
    freq_dict = {word: BoW_dict[word]/tot_words for word in BoW_dict.keys()}
    return freq_dict

In [39]:
from math import log

def inverse_document_frequency(list_of_dicts):
    tot_docs = len(list_of_dicts)
    words = set([w for w_dict in list_of_dicts for w in w_dict.keys()])
    idf_dict = {word: log(float(tot_docs)/(1.0+ sum([1 for w_dict in list_of_dicts if word in w_dict.keys()]))) for word in words}
    return idf_dict

In [40]:
def tf_idf(list_of_dicts):
    words = set([w for w_dict in list_of_dicts for w in w_dict.keys()])
    tf_idf_dicts = []
    idfs = inverse_document_frequency(list_of_dicts)
    for w_dict in list_of_dicts:
        w_dict.update({word: 0 for word in words if word not in w_dict.keys()})
        tf = term_frequency(w_dict)
        tf_idf_dicts.append({word: tf[word]*idfs[word] for word in words})
    return tf_idf_dicts

In [41]:
list_of_word_dicts = df2.select('finished_clean_lemma')

AnalysisException: "cannot resolve '`finished_clean_lemma`' given input columns: [category];;\n'Project ['finished_clean_lemma]\n+- Deduplicate [category#124]\n   +- Project [category#124]\n      +- Project [np1#419, array_union(finished_clean_lemma#264, np1_1#498) AS finished_clean_lemma#543, category#124, np1_1#498]\n         +- Project [np1#419, finished_clean_lemma#264, category#124, split(np1#419, ,) AS np1_1#498]\n            +- Union\n               :- Project [np1#419, finished_clean_lemma#264, category#124]\n               :  +- Project [article_type#0, np1#1 AS np1#419, np2#2, context#3, source#4, category#124, location#6, time#7, document#200, token#211, normalized#223, lemma#236, clean_lemma#250, finished_clean_lemma#264]\n               :     +- Project [article_type#0, np1#1, np2#2, context#3, source#4, category#124, location#6, time#7, document#200, token#211, normalized#223, lemma#236, clean_lemma#250, UDF(clean_lemma#250) AS finished_clean_lemma#264]\n               : 

In [121]:
tf_idf_list = map(lambda x: tf_idf(x), list_of_word_dicts)

In [122]:
tf_idf_dict = {c: tf_dict for c, tf_dict in zip(category, tf_idf_list)}

NameError: name 'category' is not defined

In [24]:
import sparknlp
from pyspark.ml import Pipeline
from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

spark = sparknlp.start()
print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  2.4.5
Apache Spark version:  2.4.4


In [37]:
bert = BertEmbeddings.pretrained('bert_base_cased', 'en') \
    .setInputCols(["sentence",'token'])\
    .setOutputCol("bert")\
    .setCaseSensitive(False)\
    .setPoolingLayer(0) # default 0

bert_base_cased download started this may take some time.
Approximate size to download 389.2 MB
[OK!]


In [111]:
sc.stop()

In [26]:
SparkSession._instantiatedContext = None